# Importing relevant things

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from torch import nn, optim
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Subset, TensorDataset
import os
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
import torch.nn.functional as F
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from pytorch_msssim import SSIM
from cifar_dataset import *
from ucc_dataset import *
from rcc_dataset import *
from dataloaders import *
from device_data_loader import *
from loss import *
from model import *
from params import *
from utils import *
from rcc_trainer import *
from ucc_trainer import *
from autoencoder_trainer import *


## Creating the dataset object

In [3]:
dataloaders = Dataloaders()

# RCC model local test


In [6]:
experiment2 = "ucc"
save_dir = os.path.abspath(config.weights_path)
combined_rcc_model = UCCModel().to(config.device)

#creating the trainer
rcc_trainer = UCCTrainer(experiment2, combined_rcc_model, dataloaders, save_dir)

KDE Layer initialized
UCC model initialized


In [7]:
rcc_trainer.init_params_from_checkpoint_hook(load_from_checkpoint=True, resume_steps=30001)

RuntimeError: Error(s) in loading state_dict for UCCModel:
	Unexpected key(s) in state_dict: "autoencoder.encoder.5.weight", "autoencoder.encoder.5.bias", "autoencoder.encoder.2.bias". 
	size mismatch for autoencoder.encoder.2.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([32, 2048]).
	size mismatch for autoencoder.decoder.0.weight: copying a param with shape torch.Size([128, 32]) from checkpoint, the shape in current model is torch.Size([256, 32]).
	size mismatch for autoencoder.decoder.0.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for autoencoder.decoder.2.blocks.0.conv1.weight: copying a param with shape torch.Size([256, 128, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 256, 3, 3]).
	size mismatch for autoencoder.decoder.2.blocks.0.skip_conv.weight: copying a param with shape torch.Size([256, 128, 1, 

In [ ]:
rcc_trainer.compute_min_js_divergence()